In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer

# 1. 데이터 로드 및 전처리
def load_and_preprocess(file_path):
    """
    데이터를 파일에서 불러와 사용자와 식당 데이터를 분리하고 전처리합니다.
    
    Parameters:
        file_path (str): CSV 파일 경로.

    Returns:
        tuple: 사용자 데이터, 식당 데이터, 사용자 ID, 식당 ID, 식당 원본 데이터
    """
    # 파일 로드
    data = pd.read_csv(file_path)

    # 사용자 데이터와 식당 데이터 구분
    user_data = data[data['nickname'].notna()].copy()
    restaurant_data = data[data['식당'].notna()].copy()

    # ID 컬럼 추출
    user_ids = user_data['nickname'].tolist()
    restaurant_ids = restaurant_data['식당'].tolist()

    # 문자 데이터를 제외하고 숫자 데이터만 선택
    user_data = user_data.select_dtypes(include=['float64', 'int64'])
    restaurant_data_numeric = restaurant_data.select_dtypes(include=['float64', 'int64'])

    # 결측치 처리 (평균값으로 대체)
    imputer = SimpleImputer(strategy='mean')
    user_data = imputer.fit_transform(user_data)
    restaurant_data_numeric = imputer.fit_transform(restaurant_data_numeric)

    # 정규화
    scaler = StandardScaler()
    user_data_scaled = scaler.fit_transform(user_data)
    restaurant_data_scaled = scaler.fit_transform(restaurant_data_numeric)

    return user_data_scaled, restaurant_data_scaled, user_ids, restaurant_ids, restaurant_data

# 2. PCA 수행
def perform_pca(data, n_components=3):
    """
    PCA를 실행하여 주요 성분을 추출합니다.

    Parameters:
        data (ndarray): 입력 데이터.
        n_components (int): 추출할 주성분의 개수.

    Returns:
        ndarray: PCA로 축소된 데이터.
    """
    pca = PCA(n_components=n_components)
    reduced_data = pca.fit_transform(data)
    return reduced_data

# 3. 유사도 계산
def calculate_recommendation_scores(user_features, restaurant_features):
    """
    사용자와 식당 간 유사도를 계산하여 추천 점수를 생성합니다.

    Parameters:
        user_features (ndarray): 사용자 주요 성분 데이터.
        restaurant_features (ndarray): 식당 주요 성분 데이터.

    Returns:
        ndarray: 사용자-식당 유사도 점수.
    """
    scores = cosine_similarity(user_features, restaurant_features)
    return scores

# 4. 추천 리스트 생성
def generate_recommendations(scores, user_ids, restaurant_ids, top_n=5):
    """
    사용자별로 상위 N개의 추천 식당을 생성합니다.

    Parameters:
        scores (ndarray): 유사도 점수 배열.
        user_ids (list): 사용자 ID 리스트.
        restaurant_ids (list): 식당 ID 리스트.
        top_n (int): 추천할 식당의 개수.

    Returns:
        DataFrame: 사용자별 추천 식당 리스트.
    """
    recommendations = []
    for i, user_id in enumerate(user_ids):
        top_indices = scores[i].argsort()[::-1][:top_n]
        top_restaurants = [restaurant_ids[j] for j in top_indices]
        recommendations.append({"user_id": user_id, "recommended_restaurants": top_restaurants})

    return pd.DataFrame(recommendations)

# 5. 식당 ID와 상세 정보 매핑
def map_restaurant_features(recommendations, restaurant_data):
    """
    추천된 식당 ID를 원본 데이터의 식당 이름과 특징으로 매핑합니다.

    Parameters:
        recommendations (DataFrame): 추천 결과 데이터프레임.
        restaurant_data (DataFrame): 원본 식당 데이터프레임.
    
    Returns:
        DataFrame: 식당 이름과 특징이 포함된 추천 결과.
    """
    # 식당 ID를 문자열로 변환 (추천 ID와 형식 일치)
    restaurant_data['식당'] = restaurant_data['식당'].astype(str)
    recommendations['recommended_restaurants'] = recommendations['recommended_restaurants'].apply(
        lambda ids: list(map(str, ids))
    )

    # 식당 ID와 특징 매핑
    if not restaurant_data['식당'].is_unique:
        restaurant_data = restaurant_data.drop_duplicates(subset=['식당'])
    
    id_to_features = restaurant_data.set_index('식당').to_dict('index')

    # 추천 결과 업데이트
    def map_ids_to_features(id_list):
        return [id_to_features.get(id, {"식당 이름": "Unknown", "특징": "Unknown"}) for id in id_list]

    recommendations['recommended_restaurants'] = recommendations['recommended_restaurants'].apply(map_ids_to_features)
    return recommendations

# 6. 전체 파이프라인
def main(file_path, n_components=3, top_n=5):
    """
    PCA 기반 추천 시스템 실행.

    Parameters:
        file_path (str): CSV 파일 경로.
        n_components (int): PCA 주성분의 개수.
        top_n (int): 추천할 식당의 개수.
    """
    # 데이터 로드 및 전처리
    user_data, restaurant_data, user_ids, restaurant_ids, raw_restaurant_data = load_and_preprocess(file_path)

    # PCA 수행
    user_features = perform_pca(user_data, n_components)
    restaurant_features = perform_pca(restaurant_data, n_components)

    # 추천 점수 계산
    scores = calculate_recommendation_scores(user_features, restaurant_features)

    # 추천 리스트 생성
    recommendations = generate_recommendations(scores, user_ids, restaurant_ids, top_n)

    # 식당 이름 및 특징 매핑
    recommendations = map_restaurant_features(recommendations, raw_restaurant_data)

    # 사용자별 추천 리스트를 상세 정보와 함께 출력
    for _, row in recommendations.iterrows():
        print(f"User ID: {row['user_id']}")
        print("Recommended Restaurants:")
        for restaurant in row['recommended_restaurants']:
            print(f"  - Name: {restaurant.get('식당 이름', 'Unknown')}, Features: {restaurant.get('특징', 'Unknown')}")
        print()

# 실행
file_path = "aggregated_final_result.csv"  # 업로드된 CSV 파일 경로
main(file_path, n_components=3, top_n=5)

User ID: 아암마시따
Recommended Restaurants:
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown

User ID: 밍닝몽
Recommended Restaurants:
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown

User ID: 후암동 올리
Recommended Restaurants:
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown

User ID: Junns
Recommended Restaurants:
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown
  - Name: Unknown, Features: Unknown

User ID: whrhwhdoo
Recommended Restaurants:
  - Name: Unknown, Features: Unknown
  - Name: Unknow